In [17]:
import sys
sys.path.append('..')
from src.models.pipeline import pipeline
import glob
import json
import os
import pandas as pd
import ast
from datetime import datetime
from shutil import copyfile
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
from src.models.train_model import functionsBuilder
from src.models.audio import downloadAudioFromYoutube
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
groundbase_dir = '../data/raw/groundbase'
transcripts_dir = os.path.join(groundbase_dir,'transcripts')
topic_dataset_path = os.path.join(groundbase_dir,'dataset.csv')
transcript_filespath = glob.glob(groundbase_dir + '/transcripts/*.json')

'''Read the transcript'''
transcripts_jsons = {}
for fl in transcript_filespath:
    with open(fl,encoding="utf8") as f:
        transcript =ast.literal_eval(f.read()) #json.load(f)
        vid = fl.split('\\')[-1].split('.')[0]
        #print(vid)
        transcripts_jsons[vid] = transcript
#print(transcripts_jsons)

'''Read the videos metadata to perform on them the segmentation'''
df_videos = pd.read_csv(topic_dataset_path)

''' Transfer topic shifts time to seconds units instead HH:MM:SS'''

def topic_shifts_seconds(topic_shifts):
    tp_shift_sec=[]
    for tp in topic_shifts:
        intervals = tp.split(':')
        seconds = int(intervals[2])
        minutes = int(intervals[1]) * 60
        hours = int(intervals[0]) * 60 *60
        tp_shift_sec.append(seconds + minutes + hours)
    return tp_shift_sec


for video_id in transcripts_jsons.keys():    
    df_videos.at[df_videos['video id'] == video_id,'topic shifts(ends)'] =\
    topic_shifts_seconds(\
                         df_videos[df_videos['video id'] == \
                                   video_id]['topic shifts(ends)'])

In [7]:
'''From the get optimized by bayesian we get that for the video '''
# the precision is about 66% 
vid = 'dkAr9ThdSUU'
params = {'n_clusters': 18, 'sim_thresh': 0.6, 'step_size': 49, 'window_size': 150}
workflow = 'sliding_window-tfidf-cosine-median_(3,3)-spectral_clustering'

groundbase = df_videos.loc[df_videos['video id'] == vid,'topic shifts(ends)'].values.tolist()[:-1]
transcripts = transcripts_jsons[vid]
#print(grounbase)

In [8]:
'''This running may not work at first time do not give up and run it couple of times'''

shift_times,topic_words = pipeline.run_for_baye(groundbase,transcripts,slicing_method='sliding_window',
                      window_size=params['window_size'],step_size_sd=params['step_size'],
                      #silence_threshold=-30,slice_length=1000,step_size_audio=10,wav_file_path="../../data/raw/audio/Mod-01 Lec-01 Foundation of Scientific Computing-01.wav",                
                      vector_method='tfidf',vectorizing_params=None,
                      similarity_method='cosine',
                      filter_params={"filter_type":'median',
                                     "mask_shape":(3,3),
                                     "sim_thresh":params['sim_thresh'],
                                     "is_min_thresh":True
                                     },
                     clustering_params={
                             'algorithm':'spectral_clustering',
                             'n_clusters':params['n_clusters']
                             },return_value='division')

In [14]:
len(shift_times)

15

In [16]:
[len(tp) for tp in topic_words]

[151, 141, 400, 193, 187, 196, 157, 198, 83, 155, 129, 102, 143, 113, 90, 185]

In [32]:
import heapq

In [38]:
for tp_words in topic_words:
    raw_text = ' '.join(tp_words)
    myvectorizer = CountVectorizer()
    mytf = myvectorizer.fit_transform([raw_text]).toarray()
    print(mytf)
    indexes = heapq.nlargest(3,mytf[0])
    print(indexes)
    myvectorizer.get_feature_names()[indexes]

[[2 1 1 1 1 1 1 1 1 1 1 4 3 2 1 1 1 1 2 1 1 1 1 1 2 1 1 1 3 1 1 1 3 1 1 1
  1 1 1 2 1 1 1 1 1 1 1 1 3 6 1 2 1 1 1 1 1 2 1 3 1 1 1 2 1 1 1 1 1 1 4 1
  1 1 6 7 1 3 1 2 1 1 2 1 1 2 3 2 3 7 1 2 2]]
[7, 7, 6]


TypeError: list indices must be integers or slices, not list